In [1]:
import pygame
from typing import Any, Optional, List
from pathlib import Path
from .device import Device
from .ble.charcontroller import CharController

from .input_device import InputDevice

class GameControllerInput(InputDevice):
    def __init__(
        self,
        port: Any,
        char_controllers: List[CharController],
        log_path: Optional[Path] = None,
    ):
        super().__init__(port, char_controllers, log_path)
        pygame.init()

        # Set up the game controller
        self.controller = pygame.joystick.Joystick(0)
        self.controller.init()

        # Open a file for recording the inputs
        self.file_path = log_path or Path("controller_inputs.txt")
        self.file = open(self.file_path, "w")

    def get_input(self) -> None:
        try:
            while True:
                # Check for events
                for event in pygame.event.get():
                    # Check if the event is a controller input
                    if event.type == pygame.JOYBUTTONDOWN or event.type == pygame.JOYAXISMOTION:
                        # Get the input details
                        input_type = ""
                        input_value = ""

                        if event.type == pygame.JOYBUTTONDOWN:
                            input_type = "Button"
                            input_value = event.button
                        elif event.type == pygame.JOYAXISMOTION:
                            input_type = "Axis"
                            input_value = (event.axis, event.value)

                        # Write the input to the file
                        self.file.write(f"{input_type}: {input_value}\n")

                # Exit the loop if the controller's 'A' button is pressed
                if self.controller.get_button(0) == 1:
                    break
        finally:
            # Close the file and quit pygame
            self.file.close()
            pygame.quit()

    def read_recorded_inputs(self) -> str:
        # Read and return the recorded inputs
        with open(self.file_path, "r") as file:
            inputs = file.read()
            return inputs


pygame 2.1.2 (SDL 2.0.16, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Axis: (1, -0.04626606036561175)
Axis: (1, -0.10565507980590227)
Axis: (0, 0.02850428785058138)
Axis: (0, -0.006317331461531419)
Axis: (1, -0.15186010315256204)
Axis: (0, -0.009491256447035128)
Axis: (1, -0.20465712454603718)
Axis: (0, -0.02529984435560167)
Axis: (1, -0.19476912747581407)
Axis: (0, -0.03796502578814051)
Axis: (0, -0.022156437879573963)
Axis: (1, -0.19806512649922178)
Axis: (0, -0.006317331461531419)
Axis: (1, -0.20136112552262947)
Axis: (0, 0.009491256447035128)
Axis: (0, 0.022156437879573963)
Axis: (0, 0.0348216193121128)
Axis: (0, 0.03799554429761651)
Axis: (1, -0.21787163914914395)
Axis: (0, 0.05063020722067934)
Axis: (1, -0.24097415082247384)
Axis: (0, 0.06329538865321818)
Axis: (1, -0.31028168584246346)
Axis: (0, 0.06646931363872188)
Axis: (1, -0.40269173253578294)
Axis: (0, 0.08227790154728844)
Axis: (1, -0.49839777825251014)
Axis: (0, 0.113925595873897